# Signed Graph Convolutional Network(SGCN)を用いたFraud User Detection

```sh
pip install -r /home/ubuntu/SGCN/requirements.txt
pip install torch_scatter
pip install torch_sparse
pip install easydict
```


In [ ]:
from sgcn import SignedGCNTrainer, SignedGCNPredictor
from parser_ import parameter_parser
from utils import tab_printer, read_graph, score_printer, save_logs
import easydict
import argparse
import pandas as pd
import numpy as np
import torch
import json
import networkx as nx
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import accuracy_score,confusion_matrix
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_name = input('データセット：')

In [ ]:
args = easydict.EasyDict({
        "edge_path": '/home2/kudo/SGCN/input/{0}/{0}_network.csv'.format(data_name),#'../input/{0}/user_network.csv'.format(data_name),
        "features_path":  '/home2/kudo/SGCN/input/{0}/{0}_node_feature.csv'.format(data_name),#'../input/{0}/user_network.csv'.format(data_name),
        "nodes_path": '/home2/kudo/SGCN/input/{0}/{0}_gt.csv'.format(data_name),
        "embedding_path": '/home2/kudo/SGCN/tmp/embedding/{0}_sgcn_feature05.pkl'.format(data_name), # tmp folder for cross-validation
        "regression_weights_path": '/home2/kudo/SGCN/tmp/weights/{0}_sgcn_feature05.pkl'.format(data_name),
        "inductive_model_path": None, # '/home2/kudo/SGCN/output/inductive/{0}_model'.format(data_name), # or None
        "log_path": '/home2/kudo/SGCN/logs/{0}_logs_feature05.json'.format(data_name),
        "epochs":300,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 1.0,
        "learning_rate": 0.005,  
        "weight_decay": 10e-4, 
        # "layers": [64, 32,16,8],
        "layers": [32, 16,],
        "spectral_features":False,
        "general_features": True,  
        "sample_num":None,
        "class_weights":False,
        "node_under_sampling":False,
        "hidden_residual":False,
        "eval_freq":1,
})

In [ ]:
display(pd.io.json.json_normalize(args).T)

## 10-fold cross-validation
- train : validation : test = 6:3:1

In [ ]:
#tab_printer(args)
edges, nodes_dict = read_graph(args) # nodes_dict['indice']:node_id , nodes_dict['label'] : label

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
kf = StratifiedKFold(n_splits=10)
all_indice = nodes_dict['indice']
all_labels = nodes_dict['label']
auc_scores = []

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(X=nodes_dict['indice'],y=nodes_dict['label'])):
    print("==== Training Phase ====")
    print('{}-th fold'.format(i))
    # training
    train_node_indice = all_indice[train_index]
    train_node_labels = all_labels[train_index]
    print('labels:{}'.format(np.unique(train_node_labels,return_counts=True)))
    tmp_nodes_dict = {}
    tmp_nodes_dict['all_ncount'] = nodes_dict['all_ncount']
    tmp_nodes_dict['indice'] = train_node_indice
    tmp_nodes_dict['label'] = train_node_labels
    trainer = SignedGCNTrainer(args, edges, tmp_nodes_dict)
    trainer.setup_dataset()
    trainer.create_and_train_model()
    
    if args.test_size > 0:
        # trainer.save_model() ## trainer.create_and_train_model()のなかで，すでにbest_modelが保存されている．
        # score_printer(trainer.logs)
        display(pd.DataFrame(trainer.logs['performance']))
        save_logs(args, trainer.logs)

    # test
    print("==== Test Phase ====")
    test_node_indice = all_indice[test_index]
    test_node_labels = all_labels[test_index]
    # feature = pd.read_csv(args.embedding_path,index_col='id').values
    feature = pd.read_pickle(args.embedding_path).drop('id',1).values
    test_feature = feature[test_node_indice]
    # weight = pd.read_csv(args.regression_weights_path)
    weight = pd.read_pickle(args.regression_weights_path)
    predictions = np.dot(test_feature,weight.values.T)
    probabilities = torch.nn.functional.softmax(torch.from_numpy(predictions)).numpy()
    predict_labels = probabilities.argmax(1)
    auc_score = roc_auc_score(y_true=[1 if i==-1 else 0 for i in test_node_labels],y_score=probabilities[:,1])
    auc_scores.append(auc_score)
    cmx = confusion_matrix(y_true=[1 if i==-1 else 0 for i in test_node_labels],y_pred=predict_labels)
    print("{0}-th fold's auc_score:{1}".format(i,auc_score))
    print(cmx)
    print()
    

In [ ]:
np.mean(auc_scores)

In [ ]:
_ = plt.hist(probabilities[:,1][test_node_labels==1],alpha=0.5,bins=10)
_ = plt.hist(probabilities[:,1][test_node_labels==-1],alpha=0.5,bins=10)
# plt.xlim(0,0.03)

# new_balance_theoryの結果


結果 (32,16) (non-class-weights,10e-3, test_size=0.33)
- amazon : 0.8650228
- alpha : 0.984
- otc : 0.9979548

結果 (32,16) (non-class-weights,10e-4, test_size=0.33)
- amazon :
- alpha : 0.97264735264735
- otc : 0.996

結果 (32,16, 8) (non-class-weights,10e-4, test_size=0.33)
- amazon : 0.8722951
- alpha : 
- otc : 

結果 (32,16) (non-class-weights,10e-3, test_size-0.5)
- amazon : 
- alpha : 
- otc : 

結果 (32,32,32,32) (non-class-weights,10e-3, hidden_residual,) (100 epoch)
- amazon : 0.86009158252
- alpha : 
- otc : 

結果 (32,32,32,) (class-weights,10e-3,hidden_residual) (100 epoch)
- amazon : 0.864321831
- alpha : 
- otc : 

---

結果 (32,) (no-class-weights, 0.33)
- amazon : 0.856184081
- alpha : 0.983
- otc : 0.998

結果 (32,16) (no-class-weights,test_size=0.33,weight_decay=10e-5)
- amazon : 0.871
- alpha : 0.9828
- otc : 0.997

結果 (32,16,8) (no-class-weights,test_size=0.33)
- amazon : 0.867
- alpha : 0.97123376
- otc : 0.9883699633699635

結果 (32,16,8) (no-class-weights,test_size=0.33,lamb=0)
- amazon : 0.871944
- alpha : 0.96454545
- otc : 0.994810744810

結果 (32,32,32) (no-class-weights,test_size=0.33,lamb=1.0,residual)
- amazon : 0.869
- alpha : 0.
- otc : 0.

結果 (32,16) (no-class-weights,test_size=0.33,weight_decay=10e-3)
- amazon : 0.871668935569  -->  0.866652196778459
- alpha : 0.9868181818181819 --> 0.9778071928071927
- otc : 0.9963980463980464  --> 0.99557387057

結果 (32,16) (no-class-weights,test_size=0.33,weight_decay=10e-3, amazon_unsigned)
- amazon : 
- alpha : 0.9868181818181819 --> 0.9778071928071927
- otc : 0.9963980463980464  --> 0.99557387057

---

結果 (32,16) (no-class-weights,test_size=0.66)
- amazon : 0.8635876758
- alpha : 0.9831
- otc : 0.993

結果 (64,32)
- amazon : 0.8597141032576031
- alpha : 
- otc : 

結果 (64,32,16,8)
- amazon : 0.8516649987979326
- alpha : 
- otc : 

結果
- amazon : 0.8704, (weighted classes loss : 0.871)
- alpha : (sampled: 0.9804), (normal: 0.9857)
- epinions : 
- otc : (0.9947), (normal:0.996)

## single-validation

In [ ]:
args = easydict.EasyDict({
        "edge_path": '../input/amazon/amazon_network.csv',#'../input/amazon/user_network.csv',
        "features_path":  '../input/amazon/amazon_node_feature.csv',#'../input/amazon/user_network.csv',
        "nodes_path": '../input/amazon/amazon_gt.csv',
        "embedding_path": '../tmp/embedding/amazon_sgcn_feature05.pkl', # tmp folder for cross-validation
        "regression_weights_path": '../tmp/weights/amazon_sgcn_feature05.pkl',
        "inductive_model_path": '../output/inductive/amazon_model', # or None
        "log_path": '../logs/amazon_logs_feature05.json',
        "epochs":500,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 1.0,
        "learning_rate": 0.005,  
        "weight_decay": 10e-5, 
        # "layers": [64, 32,16,8],
        "layers": [32,16],
        "spectral_features":False,
        "general_features": True,  
        "sample_num":None,
        "class_weights":False,
        "node_under_sampling":False,
})

In [ ]:
#tab_printer(args)
edges, nodes_dict = read_graph(args) # nodes_dict['indice']:node_id , nodes_dict['label'] : label

In [ ]:
trainer = SignedGCNTrainer(args, edges, nodes_dict)
trainer.setup_dataset()
trainer.create_and_train_model()

In [ ]:
if args.test_size > 0:
    trainer.save_model()
    # score_printer(trainer.logs)
    display(pd.DataFrame(trainer.logs['performance']))
    save_logs(args, trainer.logs)

#### inductive settings

In [ ]:
new_args = easydict.EasyDict({
        "edge_path": '../input/amazon_music/amazon_music_network.csv',#'../input/amazon_music/user_network.csv',
        "features_path":  '../input/amazon_music/amazon_music_node_feature.csv',#'../input/amazon_music/user_network.csv',
        "nodes_path": '../input/amazon_music/amazon_music_gt.csv',
        "embedding_path": '../tmp/embedding/amazon_music_sgcn_feature05.csv', # tmp folder for cross-validation
        "regression_weights_path": '../tmp/weights/amazon_music_sgcn_feature05.csv',
        "inductive_model_path": None, # '../output/inductive/amazon_music_model', # or None
        "log_path": '../logs/amazon_music_logs_feature05.json',
        "epochs":300,
        "test_size":0.33,
        "reduction_iterations": 128,
        "reduction_dimensions": 30,
        "seed": 42,
        "lamb": 1.0,
        "learning_rate": 0.005,  
        "weight_decay": 10e-5, 
        # "layers": [64, 32,16,8],
        "layers": [32, 16],
        "spectral_features":False,
        "general_features": True,  
        "class_weights":False,
        "sample_num":None,
        "node_under_sampling":False,
})

In [ ]:
new_edges, new_nodes_dict = read_graph(new_args)

X = np.array(pd.read_csv('../input/amazon_elec/amazon_elec_node_feature.csv')) # general node features

In [ ]:
predictor = SignedGCNPredictor(new_args, '../output/inductive/amazon_model', X, new_edges,new_nodes_dict)

In [ ]:
predictions = predictor.predict()
predict_labels = predictions.argmax(1)

In [ ]:
y_true = new_nodes_dict['label']

In [ ]:
roc_auc_score(y_true=[1 if i==-1 else 0 for i in new_nodes_dict['label']],y_score=predictions[:,1][new_nodes_dict['indice']])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
confusion_matrix([1 if i==-1 else 0 for i in new_nodes_dict['label']],predict_labels[new_nodes_dict['indice']])

In [ ]:
y_true.shape

In [ ]:
_ = plt.hist(predictions[:,1][new_nodes_dict['indice']][y_true==1],alpha=0.5,bins=10)
_ = plt.hist(predictions[:,1][new_nodes_dict['indice']][y_true==-1],alpha=0.5,bins=10)

結果
- amazon
    - elec : 0.80787063118
    - movie : 0.55
    - music : 0.76
    

In [ ]:
import json

In [ ]:
performance = pd.DataFrame(json.load(open('../logs/otc_logs_feature05.json','r'))['performance'])

performance.columns = performance.iloc[0,:]

performance = performance.iloc[1:,:]

In [ ]:
performance['AUC'].plot()

In [ ]:
feature = pd.read_csv(args.embedding_path,index_col='id')

weight = pd.read_csv(args.regression_weights_path)

In [ ]:
predictions = np.dot(feature.values,weight.values.T)

In [ ]:
probabilities = torch.nn.functional.softmax(torch.from_numpy(predictions)).numpy()

In [ ]:
predict_labels = probabilities.argmax(1)

In [ ]:
roc_auc_score(y_true=[0 if i==-1 else 1 for i in nodes_dict['label']],y_score=probabilities[:,1][nodes_dict['indice']])

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
confusion_matrix([0 if i==-1 else 1 for i in nodes_dict['label']],predict_labels[nodes_dict['indice']])

# TODO

In [ ]:
from torch_scatter import scatter_add, scatter_mean
from torch_geometric.utils import remove_self_loops, add_self_loops

In [ ]:
positive_edges = edges["positive_edges"]

In [ ]:
positive_edges = torch.from_numpy(np.array(positive_edges, dtype=np.int64).T).type(torch.long)

In [ ]:
positive_edges.shape

In [ ]:
positive_edges, _ = remove_self_loops(positive_edges, None)

In [ ]:
positive_edges.shape

In [ ]:
positive_edges = add_self_loops(positive_edges, num_nodes=nodes_dict['all_ncount'])

In [ ]:
positive_edges.shape

In [ ]:
row, col = positive_edges

In [ ]:
row,col

In [ ]:
row.shape, col.shape

In [ ]:
x = torch.from_numpy(pd.read_csv(args.features_path).values)

In [ ]:
row.shape,col.shape,x.shape

In [ ]:
x[col].shape

In [ ]:
out = scatter_mean(x[col], row, dim=0, dim_size=x.size(0))

In [ ]:
out.shape

In [ ]:
(out.numpy()==x.numpy()).all(1).sum()

In [ ]:
x.shape

In [ ]:
np.unique(row.numpy()).shape